# Friendly Adversarial Training (FAT)

In [3]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '0,1,2,3'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/fat/%s/%s' % (dataset, model_type)
num_classes = 10
lr = 0.1
momentum = 0.9
weight_decay = 2e-4
batch_size = 128
total_epochs = 120
epsilon = 8/255
alpha = 2/255
num_repeats = 10
tau = 0
omega = 0
dynamictau = True

## Inner maximization

In [3]:
def earlystop_pgd(model, xent, inputs, targets, epsilon, alpha, num_repeats, tau, omega=0):
    K = num_repeats
    count = 0
    output_target = []
    output_adv = []
    output_nat = []
    control = (torch.ones(len(targets)) * tau).cuda()
    
    noise = torch.FloatTensor(inputs.shape).uniform_(-epsilon, epsilon).cuda()
    x = torch.clamp(inputs + noise, min=0, max=1)
    
    iter_clean_data = inputs.cuda().detach()
    iter_target = targets.cuda().detach()
    output_iter_clean_data = model(inputs)
    
    while K > 0:
        x.requires_grad_()
        output = model(x)
        pred = output.softmax(dim=1).max(1, keepdim=True)[1]
        output_index = []
        iter_index = []
        
        for idx in range(len(pred)):
            if pred[idx] != iter_target[idx]:
                if control[idx] == 0:
                    output_index.append(idx)
                else:
                    control[idx] -= 1
                    iter_index.append(idx)
            else:
                iter_index.append(idx)
        
        if len(output_index) != 0:
            if len(output_target) == 0:
                output_adv = x[output_index].reshape(-1,3,32,32).cuda()
                output_nat = iter_clean_data[output_index].reshape(-1,3,32,32).cuda()
                output_target = iter_target[output_index].reshape(-1).cuda()
            else:
                output_adv = torch.cat((output_adv, x[output_index].reshape(-1,3,32,32).cuda()), dim=0)
                output_nat = torch.cat((output_nat, iter_clean_data[output_index].reshape(-1,3,32,32).cuda()), dim=0)
                output_target = torch.cat((output_target, iter_target[output_index].reshape(-1).cuda()), dim=0)
        
        loss = xent(output, iter_target)
        loss.backward()
        grads = x.grad.data
        
        if len(iter_index) != 0:
            control = control[iter_index]
            x = x[iter_index]
            iter_clean_data = iter_clean_data[iter_index]
            iter_target = iter_target[iter_index]
            output_iter_clean_data = output_iter_clean_data[iter_index]
            grads = grads[iter_index]
            eta = alpha * grads.sign()
            
            x = x.detach() + eta + omega * torch.randn(x.shape).detach().cuda()
            x = torch.min(torch.max(x, iter_clean_data-epsilon), iter_clean_data+epsilon).clamp(min=0, max=1)
            count += len(iter_target)
        else:
            output_adv = output_adv.detach()
            return output_adv, output_target, output_nat, count
        
        K = K - 1
    
    if len(output_target) == 0:
        output_target = iter_target.reshape(-1,3,32,32).cuda()
        output_adv = x.reshape(-1,3,32,32).cuda()
        output_nat = iter_clean_data.reshape(-1,3,32,32).cuda()
    else:
        output_adv = torch.cat((output_adv, x.reshape(-1,3,32,32)), dim=0).cuda()
        output_target = torch.cat((output_target, iter_target.reshape(-1)), dim=0).squeeze().cuda()
        output_nat = torch.cat((output_nat, iter_clean_data.reshape(-1,3,32,32).cuda()), dim=0).cuda()
    output_adv = output_adv.detach()
    return output_adv, output_target, output_nat, count

## Training (Outer minimization)

In [4]:
def training(epoch, model, dataloader, optimizer, num_classes,
             tau, epsilon=8/255, alpha=2/255, num_repeats=10):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
        
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        x_adv, y, x_nat, count  = earlystop_pgd(model, xent, inputs, targets, epsilon, alpha, num_repeats, tau, omega=0)
        logits = model(x_adv)
        loss = xent(logits, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits.data, dim=1).eq(y.data).cpu().sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), total_loss/len(dataloader),
                     num_correct/batch, total_correct/total))

In [5]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
        
            noise = torch.FloatTensor(inputs.shape).uniform_(-epsilon,epsilon).cuda()
            x = torch.clamp(inputs+noise, min=0, max=1)
            for _ in range(num_repeats):
                x.requires_grad_()
                logits = model(x)
                loss = xent(logits, targets)
                loss.backward()
                grads = x.grad.data
                x = x.detach() + alpha*torch.sign(grads).detach()
                x = torch.min(torch.max(x, inputs - epsilon), inputs + epsilon).clamp(min=0, max=1)
        
            with torch.no_grad():
                logits_nat = model(inputs)
                logits_adv = model(x)
        
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
        
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [6]:
def adjust_tau(epoch, dynamictau, tau):
    if dynamictau:
        if epoch <= 50:
            tau = 0
        elif epoch <= 90:
            tau = 1
        else:
            tau = 2
    return tau

def adjust_learning_rate(optimizer, epoch, lr):
    """decrease the learning rate"""
    if epoch >= 60:
        lr = lr * 0.1
    if epoch >= 90:
        lr = lr * 0.01
    if epoch >= 110:
        lr = lr * 0.005
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD(model.parameters(),lr=lr, momentum=momentum, weight_decay=weight_decay)

#scheduler = [int(total_epochs*0.5), int(total_epochs*0.75)]
#adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(total_epochs):
    training(epoch, model, train_dataloader, optimizer, num_classes, 
             adjust_tau(epoch, dynamictau=dynamictau, tau=tau), epsilon, alpha, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats)
        
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate(optimizer, epoch, lr)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/383] | loss: 2.3641 (avg: 0.0062) | acc: 0.0078 (avg: 0.0078) |
Epoch 0 [100/383] | loss: 1.7517 (avg: 0.5451) | acc: 0.3125 (avg: 0.2057) |
Epoch 0 [200/383] | loss: 1.7780 (avg: 1.0135) | acc: 0.2812 (avg: 0.2451) |
Epoch 0 [300/383] | loss: 1.7770 (avg: 1.4558) | acc: 0.2422 (avg: 0.2680) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:34<00:00,  4.65it/s, acc_nat=0.475, acc_rob=0.138]


Epoch 1 [0/383] | loss: 1.6750 (avg: 0.0044) | acc: 0.3359 (avg: 0.3359) |
Epoch 1 [100/383] | loss: 1.4029 (avg: 0.4104) | acc: 0.4297 (avg: 0.3555) |
Epoch 1 [200/383] | loss: 1.5228 (avg: 0.7964) | acc: 0.3828 (avg: 0.3707) |
Epoch 1 [300/383] | loss: 1.3119 (avg: 1.1661) | acc: 0.4531 (avg: 0.3801) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:39<00:00,  4.56it/s, acc_nat=0.53, acc_rob=0.204]


Epoch 2 [0/383] | loss: 1.3621 (avg: 0.0036) | acc: 0.3906 (avg: 0.3906) |
Epoch 2 [100/383] | loss: 1.3016 (avg: 0.3405) | acc: 0.4375 (avg: 0.4501) |
Epoch 2 [200/383] | loss: 1.0360 (avg: 0.6681) | acc: 0.5938 (avg: 0.4615) |
Epoch 2 [300/383] | loss: 1.1709 (avg: 0.9925) | acc: 0.5312 (avg: 0.4651) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [03:23<00:00,  4.90it/s, acc_nat=0.87, acc_rob=0.399]


Epoch 118 [0/383] | loss: 0.1283 (avg: 0.0003) | acc: 0.9531 (avg: 0.9531) |
Epoch 118 [100/383] | loss: 0.1334 (avg: 0.0552) | acc: 0.9531 (avg: 0.9082) |
Epoch 118 [200/383] | loss: 0.1424 (avg: 0.1210) | acc: 0.9375 (avg: 0.8984) |
Epoch 118 [300/383] | loss: 0.1624 (avg: 0.1823) | acc: 0.9375 (avg: 0.8971) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:23<00:00,  4.93it/s, acc_nat=0.892, acc_rob=0.462]


Epoch 119 [0/383] | loss: 0.3391 (avg: 0.0009) | acc: 0.8438 (avg: 0.8438) |
Epoch 119 [100/383] | loss: 0.1330 (avg: 0.0539) | acc: 0.9766 (avg: 0.9112) |
Epoch 119 [200/383] | loss: 0.2742 (avg: 0.1117) | acc: 0.8594 (avg: 0.9058) |
Epoch 119 [300/383] | loss: 0.1904 (avg: 0.1704) | acc: 0.9219 (avg: 0.9030) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:23<00:00,  4.92it/s, acc_nat=0.891, acc_rob=0.446]
